## TEXT CLASSIFICATION

Họ tên và MSSV: Huỳnh Chí Phi Thuận, KHDL2211038

Lớp: khdl2211 Ngày học: 4/10/2025 

In [15]:
import collections
import nltk
import os
import random
nltk.download('punkt_tab')
nltk.download('wordent')


import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_word = set(stopwords.words('english'))

stop_word = {'ourselves', 'hers', 'between', 'yourself', 'but', 'again',
'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they',
'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into',
'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as'
'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we',
'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more'
'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above',
'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any',
'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does'
'yourselves', 'then', 'that', 'because', 'what', 'over', 'why',
'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself',
'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those',
'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my',
'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was',
'here', 'than'}

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Error loading wordent: Package 'wordent' not found in
[nltk_data]     index
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def load_file(directory):
    result = []  # Tạo một danh sách rỗng để chứa nội dung của các file

    # Duyệt qua từng tên file trong thư mục được truyền vào
    for fname in os.listdir(directory):

        # Mở từng file ở chế độ đọc, dùng mã hóa ISO-8859-1 (hợp với dữ liệu Enron)
        with open(directory + '/' + fname, 'r', encoding='ISO-8859-1') as f:
            result.append(f.read())  # Đọc toàn bộ nội dung file và thêm vào danh sách

    return result  # Trả về danh sách chứa nội dung của tất cả các file


In [16]:
def preprocess_sentence(sentence):
    # Khởi tạo bộ lemmatizer của NLTK để đưa từ về gốc (run, runs → run)
    lematizer = nltk.WordNetLemmatizer()

    # Tách câu thành từng từ (tokens)
    processed_tokens = nltk.word_tokenize(sentence)

    # Chuyển tất cả từ thành chữ thường để chuẩn hóa (Avoid → avoid)
    processed_tokens = [w.lower() for w in processed_tokens]

    # Đếm số lần xuất hiện của từng từ
    word_counts = collections.Counter(processed_tokens)

    # Lấy ra những từ xuất hiện ít nhất (uncommon words)
    # most_common()[:-10:-1] = 9 từ ít phổ biến nhất (nhưng cách viết này khá mơ hồ)
    uncommon_words = word_counts.most_common()[:-10:-1]

    # Bước lọc 1: Loại bỏ stop words (các từ phổ biến như: the, is, and, ...)
    processed_tokens = [w for w in processed_tokens if w not in stop_word]

    # Bước lọc 2: Loại bỏ từ ít xuất hiện (uncommon words)
    processed_tokens = [w for w in processed_tokens if w not in uncommon_words]

    # Lemmatize: đưa từ về gốc ngữ nghĩa (better → good, running → run)
    processed_tokens = [lematizer.lemmatize(w) for w in processed_tokens]

    # Trả về danh sách token đã xử lý
    return processed_tokens


In [8]:
def feature_extraction(tokens):
    '''
    Mô tả:
    - Hàm này chuyển mỗi từ (token) thành một đặc trưng (feature).
    - Giá trị của đặc trưng là số lần từ đó xuất hiện trong câu.
    - Kết quả trả về là một dictionary dạng: {'word1': count1, 'word2': count2, ...}
    '''

    # collections.Counter sẽ đếm số lần xuất hiện của mỗi từ trong danh sách tokens
    # dict(...) chuyển kết quả Counter thành dictionary bình thường
    return dict(collections.Counter(tokens))

In [9]:
def train_test_split(dataset, train_size=0.8):
    # Tính số lượng mẫu dùng cho tập huấn luyện dựa vào tỷ lệ train_size
    num_training_examples = int(len(dataset) * train_size)

    # Trả về hai phần: 
    # - phần đầu là tập huấn luyện (training set)
    # - phần sau là tập kiểm tra (test set)
    return dataset[:num_training_examples], dataset[num_training_examples:]


In [17]:
import nltk
import random
import collections

# ===== 1. Đọc dữ liệu từ thư mục =====
# positive: spam, label = 1
# negative: ham, label = 0

positive_examples = load_file('data\ham')
negative_examples = load_file('data\spam')

# ===== 2. Tiền xử lý mỗi email =====
# Biến danh sách chuỗi thành danh sách token sau khi xử lý (lower, stopword, lemmatize,...)
positive_examples = [preprocess_sentence(email) for email in positive_examples]
negative_examples = [preprocess_sentence(email) for email in negative_examples]

# ===== 3. Gắn nhãn (label) cho từng email =====
positive_examples = [(email, 1) for email in positive_examples]  # spam = 1
negative_examples = [(email, 0) for email in negative_examples]  # ham = 0

# ===== 4. Gộp và xáo trộn toàn bộ dữ liệu =====
all_examples = positive_examples + negative_examples
random.shuffle(all_examples)

# In ra số lượng email đã xử lý
print('{} emails processed.'.format(len(all_examples)))

# ===== 5. Trích xuất đặc trưng từ mỗi email =====
# Tạo bộ dữ liệu gồm (features, label) theo định dạng của NLTK NaiveBayesClassifier
featurized = [(feature_extraction(email), label) for email, label in all_examples]

# ===== 6. Chia dữ liệu train/test =====
training_set, test_set = train_test_split(featurized, train_size=0.7)

# ===== 7. Huấn luyện mô hình Naive Bayes (NLTK) =====
model = nltk.classify.NaiveBayesClassifier.train(training_set)

# ===== 8. Đánh giá trên tập huấn luyện =====
training_error = nltk.classify.accuracy(model, training_set)
print('✅ Accuracy on training set: {:.2f}'.format(training_error * 100))

# ===== 9. Đánh giá trên tập kiểm tra =====
testing_error = nltk.classify.accuracy(model, test_set)
print('✅ Accuracy on test set: {:.2f}'.format(testing_error * 100))


<>:9: SyntaxWarning: invalid escape sequence '\h'
<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\h'
<>:10: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Admin\AppData\Local\Temp\ipykernel_18080\1026088342.py:9: SyntaxWarning: invalid escape sequence '\h'
  positive_examples = load_file('data\ham')
C:\Users\Admin\AppData\Local\Temp\ipykernel_18080\1026088342.py:10: SyntaxWarning: invalid escape sequence '\s'
  negative_examples = load_file('data\spam')


5172 emails processed.
✅ Accuracy on training set: 96.10
✅ Accuracy on test set: 93.88


Dưới đây là phần trả lời chi tiết cho từng câu hỏi của bạn liên quan đến tiền xử lý văn bản và mô hình phân loại (ví dụ như email spam):

---

### ✅ **1. Mục đích của việc sử dụng `WordNetLemmatizer` trong tiền xử lý văn bản là gì?**

**WordNetLemmatizer** giúp đưa một từ về **dạng gốc (lemma)** của nó, ví dụ:
- "running" → "run"
- "better" → "good"

👉 Mục đích:
- **Giảm số lượng từ khác nhau nhưng cùng gốc**, giúp mô hình học hiệu quả hơn.
- Tăng **tính tổng quát** và **giảm nhiễu** trong tập dữ liệu.
- Cải thiện độ chính xác khi trích xuất đặc trưng.

---

### ✅ **2. Trong đoạn mã `train_test_split`, tham số `train_size=0.8` có ý nghĩa gì?**

`train_size=0.8` có nghĩa là:
- 80% dữ liệu sẽ được dùng để **huấn luyện mô hình**.
- 20% còn lại dùng để **kiểm tra và đánh giá** mô hình.

👉 Đây là tỷ lệ phổ biến để đảm bảo mô hình học tốt mà vẫn được kiểm tra khách quan.

---

### ✅ **3. Tại sao chúng ta cần xáo trộn dữ liệu trước khi chia tập huấn luyện và kiểm tra?**

Nếu dữ liệu được sắp xếp theo thứ tự (ví dụ: tất cả email spam ở đầu, không spam ở cuối), thì:
- **Tập huấn luyện** có thể chỉ chứa một loại dữ liệu → mô hình học không đủ thông tin.
- Dễ bị **overfitting** hoặc **underfitting**.

👉 Việc **xáo trộn (shuffle)** đảm bảo:
- Hai tập (train/test) đều đại diện đầy đủ cho các loại dữ liệu khác nhau.
- Mô hình **tổng quát tốt hơn** khi áp dụng vào dữ liệu thực tế.

---

### ✅ **4. Mô hình sử dụng thuật toán nào để phân loại email spam?**

Thông thường trong các ví dụ đơn giản, mô hình phân loại email spam sử dụng thuật toán:
🔸 **Naive Bayes (Multinomial Naive Bayes)**

- Đây là thuật toán thường dùng cho **phân loại văn bản** do:
  - Hiệu quả cao
  - Tính toán nhanh
  - Phù hợp với dữ liệu biểu diễn dưới dạng tần suất từ

---

### ✅ **5. Nếu muốn cải thiện độ chính xác của mô hình, bạn có thể thực hiện những thay đổi nào?**

Một số cách phổ biến:
- **Tiền xử lý nâng cao**: loại bỏ stop words, lemmatization/stemming, chuẩn hóa văn bản.
- **Tối ưu vector hóa**:
  - Dùng TF-IDF thay vì CountVectorizer.
  - Sử dụng embedding (Word2Vec, BERT) nếu cần độ chính xác cao.
- **Tăng dữ liệu huấn luyện**.
- **Cân bằng dữ liệu** nếu bị lệch nhãn (spam/ham không đồng đều).
- **Tối ưu mô hình**:
  - Thử các thuật toán khác: SVM, Random Forest, XGBoost.
  - Điều chỉnh siêu tham số (hyperparameter tuning).
- **Loại bỏ nhiễu/ngôn ngữ không cần thiết** trong dữ liệu.

---

### ✅ **6. Vì sao Naive Bayes lại phù hợp với bài toán phân loại văn bản?**

🔹 Naive Bayes phù hợp vì:
- **Giả định độc lập** giữa các đặc trưng (từ ngữ) — đơn giản nhưng hiệu quả trong thực tế.
- **Hiệu suất nhanh**, phù hợp với dữ liệu văn bản lớn.
- **Ít tài nguyên tính toán**.
- Đặc biệt hiệu quả khi đặc trưng là tần suất từ hoặc TF-IDF.

👉 Mặc dù giả định độc lập là đơn giản hoá, nhưng vẫn cho kết quả rất tốt trong các bài toán như:
- **Phân loại email spam**
- **Phân loại tin tức, đánh giá sentiment (cảm xúc)**

---

Nếu bạn đang làm bài tập hoặc dự án cụ thể, mình có thể giúp mở rộng từng phần theo hướng đó luôn nhé!